# 第10回 アルゴリズム入門：問題の複雑さ

___
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tsuboshun/begin-python/blob/gh-pages/_sources/workbook/lecture10.ipynb)

___

## この授業で学ぶこと

前回の授業で挿入ソートとクイックソートを紹介し、それらの実行速度に差があることを見た。特にデータ数を増やしたときの実行時間の増え方に大きな差があることを見た。今回は、アルゴリズムの実行時間を見積もるための考え方である計算量について学び、先週の結果について理解を深める。また、それをもとにP・NPという問題の難しさを表す分類について学ぶ。

これらの話題をきちんと学ぶためには数理的な議論が必要になるが、このテキストではなるべく数理的な話には立ち入らず直感的な説明を心がけている。より詳しく学びたい学生は、他の文献を当たってみてほしい。

### 準備

In [ ]:
import time
import random
import itertools

## 計算量

### 計算量とは

**計算量**（computational complexity）とは、計算ステップ数がデータ数 $N$ に対してどのように増えるかを表す指標である。
計算量はアルゴリズムの実行時間を見積もるために使われる。

例として、次の2つのプログラムの実行時間を考える。

In [ ]:
def single_loop(N):
    count = 0
    for i in range(N):
        count += 1
    return count

In [ ]:
def double_loop(N):
    count = 0
    for i in range(N):
        for j in range(N):
            count += 1
    return count

引数 `N` でループ回数を受け取り、`simple_loop()` は一重ループ、`double_loop()` は二重ループで単純な足し算を実行する。
これらの関数の実行時間を次の `exec_time()` で測定してみよう。

In [ ]:
def exec_time(func, N):
    start = time.time()
    func(N)
    end = time.time()
    return end - start

In [ ]:
N = 10 ** 3
print(f"single_loopの実行時間： {exec_time(single_loop, N)}")
print(f"double_loopの実行時間: {exec_time(double_loop, N)}")

```{admonition} 関数もオブジェクト
:class: note
第4回の授業で「Pythonに出てくるほとんど全ての要素はオブジェクトとして作られている」と説明したが、関数も例外ではない。関数もオブジェクトであり、一種のデータとして扱うことができる。したがって、その他のデータと同じく変数に代入することもできる。
<pre>func = single_loop
func(10**2)  # これはsingle_loop(10**2)と同じ！</pre>
`exec_time()` では引数 `func` で関数を受け取り、`exec_time()` の内部で `func(N)` と実行している。このようなことが可能なのも、関数がオブジェクトだからである。
```

さて、データ数と実行時間の関係を調べてみると、私の環境（Google Colab）では次のようになった。

| `N`  |  `single_loop(N)` の実行時間（秒） |  `double_loop(N)` の実行時間（秒） | 
| ---- | ---- | ---- |
| 100  |  0.00000429  | 0.000491 |
| 1000  |  0.0000438  | 0.0296 |
| 10000  |  0.000513  | 4.00 | 
| 100000  |  0.00373  | 344 | 

$N$ が10倍になると、`single_loop(N)` の実行時間はおよそ10倍、`double_loop(N)` の実行時間はおよそ100倍に増えていることがわかる。

これは次のように理解できる。`single_loop(N)` では `count = 0` を1回実行し、`count += 1` を $N$ 回実行している。$N$ が十分大きいとき、全体の計算ステップ数はほぼ $N$ に比例するので、$N$ が10倍になると実行時間もおよそ10倍になる。一方で、`double_loop(N)` では `count = 0` を1回実行し、`count += 1` を $N^2$ 回実行している。$N$ が十分大きいとき、全体の計算ステップ数はほぼ $N^2$ に比例するので、$N$ が10倍になると実行時間はおよそ100倍になる。

このようなデータ数と計算ステップ数の関係について、 `single_loop(N)` の計算量は $O(N)$ （読み方：オーダー$N$）、`double_loop(N)` の計算量は $O(N^2)$ であると言う。

計算量がわかると、大まかな実行時間の見積もりができる。例えば、`double_loop(N)` は $N = 10^5$ のとき $6$ 分くらいかかっているので、その $10$ 倍の $N = 10^6$ のとき $6\times 10^2 = 600$ 分（つまり $10$ 時間！）くらいかかるだろうと見積もることができる。表の結果を踏まえると、一般的には計算ステップ数 $10^7\sim10^8$ につき $1$ 秒程度かかると見積もると良いと思われる。もちろんこれはプログラムの内容やアルゴリズム、実行環境にも依るので、あくまで一つの目安として捉えてほしい。

### オーダー記法について

さて、計算量のオーダー記法について次の2点を補足する。
1. オーダー記法では定数倍の寄与は考えない
2. オーダー記法では低次の項の寄与は考えない  

1つ目については、例えば計算ステップ数が $2N$ であったとしたとしても、オーダー記法としては $O(N)$ と書くということである。
これは定数倍があったとしても、データ数 $N$ が増えたときの計算量の増え方には関係がないからである。つまり、データ数が $N'$ に増えたとして、計算ステップ数の比を考えると $(cN') / (cN) = N' / N$ というように定数倍 $c$ はあってもなくても変わらない。

2つ目については、例えば計算ステップ数が $N^2 + N$ であったとしても、オーダー記法としては $O(N^2)$ と書くということである。
これは実行時間が問題になるような $N$ の大きい領域では、低次の $O(N)$ の寄与はほとんど無視できるからである。
例えば、`single_loop(N)` と `double_loop(N)` の実行時間を比較した表で両者を足したとしても、 `single_loop(N)` の実行時間が無視できるほど小さいため `double_loop(N)` の値とほとんど変わらない。

## 巡回セールスマン問題

In [ ]:
def generate_distances(N):
    ret = [[0] * N for i in range(N)]  # N * N の 2次元リストの用意
    for i in range(N):
        for j in range(N):
            if i < j:
                d = random.randint(1, 10)  # 1以上10以下の整数をランダムに生成
                ret[i][j] = d
                ret[j][i] = d
    return ret


def solve_tsp(distances):
    # 都市を回る順番について、全ての組み合わせを生成する
    city_combinations = itertools.permutations(range(len(distances)))

    # 最短の経路と距離を初期化する
    shortest_distance = 10**8  # 十分に大きい値であれば何でも良い
    shortest_path = None

    # 全ての経路を試す
    for path in city_combinations:
        total_distance = 0
        for i in range(len(path)-1):
            total_distance += distances[path[i]][path[i+1]]
        # 最短経路を更新する
        if total_distance < shortest_distance:
            shortest_distance = total_distance
            shortest_path = path

    # 結果を出力する
    print("最短距離:", shortest_distance)
    print("最短経路:", shortest_path)

In [ ]:
N = 8

# 都市間の距離を表す2次元リスト
distances = generate_distances(N)

# 最短経路の探索
start = time.time()
solve_tsp(distances)
end = time.time()

print(f"実行時間: {end - start:.4f}秒")